In [1]:
from playwright.async_api import async_playwright
# Update python path to include the parent directory
import os
import sys
import datetime
from datetime import timedelta

from urllib import parse

sys.path.append("..")

os.environ["ROOT_PATH"] = "/Users/aennassiri/Projects/Personal/ticket-washer"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/Users/aennassiri/Projects/Personal/ticket-washer/configuration/fubloo-app-1f213ca274de.json"
)

In [2]:
import requests

import pandas as pd
from src.services.leads import get_leads, get_last_lead

In [3]:

pw = await async_playwright().start()
browser = await pw.chromium.launch(headless=False)
# browser = await pw.chromium.launch()


In [4]:
import os
os.makedirs('playwright/.auth', exist_ok=True)
if os.path.exists('playwright/.auth/state.json'):
    context = await browser.new_context(storage_state="playwright/.auth/state.json")
    page = await context.new_page()


In [5]:
async def extract_info(page):
    container = await page.query_selector('.e1usz6i03')
    element = await container.query_selector('#phone-number-content')
    phone_number  = await element.inner_text()
    print(phone_number)
    element = await container.query_selector("#email-address-content")
    email_address = await element.inner_text()
    print(email_address)
    element = await container.query_selector("#past-addresses-content")
    past_addresses = await element.inner_text()
    print(past_addresses)
    element = await container.query_selector("#current-address-content")
    current_address = await element.inner_text()
    print(current_address)
    element = await container.query_selector("#summary-name")
    summary_name = await element.inner_text()
    print(summary_name) 
    return {
        "phone_number": phone_number,
        "email_address": email_address,
        "past_addresses": past_addresses,
        "current_address": current_address,
        "summary_name": summary_name
    }

In [6]:
async def search_person(page, firstName, lastName, middleName=None, age=None,
    city=None,
    state=None,
    zip=None,
    addressLine1=None,
    addressLine2=None):
    url = f"https://www.spokeo.com/{firstName}-{lastName}"
    params = {}
    if middleName is not None:
        params["middle_name"] = middleName
    if age is not None:
        # age ranges: 18-25; 26-39; 40-49; 50-59; 60+
        if age < 18:
            raise ValueError("Age must be greater than 18")
        if age < 26:
            params["age_range"] = "18-25"
        elif age < 40:
            params["age_range"] = "26-39"
        elif age < 50:
            params["age_range"] = "40-49"
        elif age < 60:
            params["age_range"] = "50-59"
        else:
            params["age_range"] = "60+"
    if city is not None:
        params["city"] = city
    if state is not None:
        params["state"] = state
    if params:
        url = f"{url}?{parse.urlencode(params)}"

    print(url)
    await page.goto(url)    

    link_elements = await page.query_selector_all('.e1ndw42t0')
    # links = [
    #     link_element.get_attribute('href')
    #     for link_element in link_elements
    # ]
    links = []
    for link_element in link_elements:
        link = await link_element.get_attribute('href')
        links.append(f"https://www.spokeo.com{link}")

    results = []
    for link in links:
        await page.goto(link)
        try:
            output = await extract_info(page)
            results.append(output)
        except Exception as e:
            print(e,"==>", link)
            pass

    return results

# Leads Analysis


In [7]:
today = datetime.datetime.now()

leads_not_found = get_last_lead(
    start_date=today - timedelta(days=5),
    end_date=today + timedelta(days=1),
    status="new",
    limit=1000,
)

df = pd.DataFrame([lead.model_dump() for lead in leads_not_found])


df["middle_name"] = df["middle_name"].fillna("")
df["first_name"] = df["first_name"].fillna("")
df["last_name"] = df["last_name"].fillna("")
df["state"] = df["state"].fillna("MO")

/Users/aennassiri/Projects/Personal/ticket-washer/src/services/leads.py:145: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  leads_list = leads_list.where("case_date", ">=", start_date)
/Users/aennassiri/Projects/Personal/ticket-washer/src/services/leads.py:149: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  leads_list = leads_list.where("case_date", "<=", end_date)
/Users/aennassiri/Projects/Personal/ticket-washer/src/services/leads.py:151: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  leads_list = leads_list.where("status", "==", status)


In [8]:
df

,id,case_id,first_name,last_name,middle_name,court_code,age,year_of_birth,email,phone,...,violation,court,accidentCheckbox,commercialDriverLicence,ticket_img,user_id,report,details,lead_source,cloudtalk_upload
0,None,240074719,DEVIN,CULKIN,MICHEAL,CT21,27.0,1996.0,,None,...,None,None,False,False,240074719.png,None,None,,None,False
1,None,191003505,BRIAN,RUZICKA,JUSTIN,CT21,34.0,1989.0,,None,...,None,None,False,False,191003505.png,None,None,,None,False
2,None,230397023,ANGEL,LADD,LASSHA,CT21,19.0,2005.0,,None,...,None,None,False,False,230397023.png,None,None,,None,False
3,None,160466473,ERIC,BOLDEN,LEWIS,CT21,40.0,1983.0,,None,...,None,None,False,False,160466473.png,None,None,,None,False
4,None,240074722,DANIEL,KNIGHT,JAMES,CT21,34.0,1989.0,,None,...,None,None,False,False,240074722.png,None,None,,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,None,170692789,ADRIAN,HUDSON,A.,CT21,38.0,1985.0,,None,...,None,None,False,False,170692789.png,None,None,,None,False
683,None,170692787,STEVEN,STRINGER,LEE,CT21,38.0,1985.0,,None,...,None,None,False,False,170692787.png,None,None,,None,False
684,None,151327257,MICHAEL,FOULK,S,SMPDB0001_CT32,17.0,2007.0,,None,...,None,None,False,False,151327257.png,None,None,,None,False
685,None,150773572,CHAUNCEY,ALLER,L,SMPDB0001_CT04,17.0,2006.0,,None,...,None,None,False,False,150773572.png,None,None,,None,False


In [13]:
results = {}

for index, row in df.iterrows():
    print(f"Looking for {row['case_id']}")
    print(
        row["first_name"],
        row["last_name"],
        row["middle_name"],
        row["age"],
        row["city"],
        row["state"],
        row["zip_code"],
        row["address"],
    )
    results[row["case_id"]] = await search_person(
        page,
        row["first_name"],
        row["last_name"],
        row["middle_name"],
        row["age"],
        row["city"],
        row["zip_code"],
        row["state"],
        row["address"],
    )

Looking for 240074719
DEVIN CULKIN MICHEAL 27.0 None MO None None
https://www.spokeo.com/DEVIN-CULKIN?middle_name=MICHEAL&age_range=26-39
See available information
See available information
1100 Rue Claudine Bonne Terre, MO
10494 Shut in Rd, Irondale, MO
Devin M Culkin, Age 27
Looking for 191003505
BRIAN RUZICKA JUSTIN 34.0 None MO None None
https://www.spokeo.com/BRIAN-RUZICKA?middle_name=JUSTIN&age_range=26-39
See available information
briankb420@msn.com
7012 Hunters Rdg House Springs, MO +1 more
3884 Swiss Dr, Apt 5, Saint Louis, MO
Brian Justin Ruzicka Sr, Age 34
(417) 756-2388 +1 phone
waterinhole@gmail.com +2 emails
1127 W Elm St Morrisville, MO +6 more
5495 Highway H, Pleasant Hope, MO
Brian Michael Ruzicka, Age 38
(760) 596-1355 +2 phones
See available information
5255 Castle Rd La Canada Flintridge, CA
15826 Bear Valley Rd, Victorville, CA
Brian Ruzicka, Age 34
(314) 894-1529 +1 phone
leslieruzicka@hotmail.com +1 email
9880 NE Decora Ln, Apt 1137 Hillsboro, OR +4 more
5727 Gre

ValueError: Age must be greater than 18

In [14]:
results

{'240074719': [{'phone_number': 'See available information',
   'email_address': 'See available information',
   'past_addresses': '1100 Rue Claudine Bonne Terre, MO',
   'current_address': '10494 Shut in Rd, Irondale, MO',
   'summary_name': 'Devin M Culkin, Age 27'}],
 '191003505': [{'phone_number': 'See available information',
   'email_address': 'briankb420@msn.com',
   'past_addresses': '7012 Hunters Rdg House Springs, MO +1 more',
   'current_address': '3884 Swiss Dr, Apt 5, Saint Louis, MO',
   'summary_name': 'Brian Justin Ruzicka Sr, Age 34'},
  {'phone_number': '(417) 756-2388 +1 phone',
   'email_address': 'waterinhole@gmail.com +2 emails',
   'past_addresses': '1127 W Elm St Morrisville, MO +6 more',
   'current_address': '5495 Highway H, Pleasant Hope, MO',
   'summary_name': 'Brian Michael Ruzicka, Age 38'},
  {'phone_number': '(760) 596-1355 +2 phones',
   'email_address': 'See available information',
   'past_addresses': '5255 Castle Rd La Canada Flintridge, CA',
   'cu

In [12]:
results

[]

In [270]:
await browser.close()
await pw.stop()